In [1]:
import pandas as pd
import numpy as np
import sys
from numpy import NaN
from scipy import stats
import statistics
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from scipy import signal
from scipy import linalg
from sklearn.decomposition import FastICA, PCA
#from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline  
%matplotlib notebook
%pylab

#pca = PCA(n_components=3)
#pca_red = PCA(n_components=3)

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['linalg']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# uncomment this if you want to see all the dataframe
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

In [3]:
cells = ["TrkA","TrkB"]
treatments = ["DMSO","PI3Kinh","JNKinh","S6Kinh","RSKinh","MEKinh","AKTinh","JNK8inh"]
treatments_1 = ["PI3Kinh","JNKinh","S6Kinh","RSKinh"]
treatments_2 = ["MEKinh","AKTinh","JNK8inh"]
treatments_actual = ["JNKinh","S6Kinh","RSKinh","MEKinh","AKTinh","JNK8inh"]
replicates_DMSO = 6
replicates_rest = 3

In [4]:
RPPA_total = pd.read_csv('RPPA_data_trusted.csv').set_index("Antibodies")
#RPPA_total

# Normalizing by GAPDH
GAPDH_data = RPPA_total.loc["GAPDH"]
RPPA_total_norm = RPPA_total.divide(GAPDH_data)
RPPA_total_norm = RPPA_total_norm.drop(index="GAPDH")
#RPPA_total_norm.filter(regex='.*TrkA.*DMSO.*')

#RPPA_total_norm

In [5]:
# normalizing by 0 DMSO time point
for cell in cells:
    for treatment in treatments:
        if treatment == "DMSO":
            replicates = replicates_DMSO
        else:
            replicates = replicates_rest
        for replicate_ in range(replicates):
            if treatment in treatments_1:
                replicate = replicate_ + 1
            elif treatment in treatments_2:
                replicate = replicate_ + 4
            else:
                replicate = replicate_ + 1
            #data_row_0 = cell + "_" + treatment + "_" + "0" + "_r" + str(replicate)
            data_row_0 = cell + "_DMSO_0_r" + str(replicate)
            data_row_time = cell + "_" + treatment + "_" + "10" + "_r" + str(replicate)
            RPPA_total_norm[data_row_time] = RPPA_total_norm[data_row_time] / RPPA_total_norm[data_row_0]
            data_row_time = cell + "_" + treatment + "_" + "45" + "_r" + str(replicate)
            RPPA_total_norm[data_row_time] = RPPA_total_norm[data_row_time] / RPPA_total_norm[data_row_0]
            #del RPPA_total_norm[data_row_0]

for cell in cells:
    replicates = replicates_DMSO
    for replicate_ in range(replicates):
        replicate = replicate_ + 1
        #data_row_0 = cell + "_" + treatment + "_" + "0" + "_r" + str(replicate)
        data_row_0 = cell + "_DMSO_0_r" + str(replicate)
        RPPA_total_norm[data_row_0] = RPPA_total_norm[data_row_0] / RPPA_total_norm[data_row_0]
        #del RPPA_total_norm[data_row_0]


# reading data on Trk signalling
Trk_norm = pd.read_csv('RPPA_data_Trk_normalized_new.csv').set_index("Antibodies")
RPPA_total_norm = RPPA_total_norm.append(Trk_norm)
RPPA_total_norm
#RPPA_total_norm.to_csv("RPPA_data_trusted_normalized.csv")

,TrkA_DMSO_0_r1,TrkA_DMSO_0_r2,TrkA_DMSO_0_r3,TrkA_DMSO_10_r1,TrkA_DMSO_10_r2,TrkA_DMSO_10_r3,TrkA_DMSO_45_r1,TrkA_DMSO_45_r2,TrkA_DMSO_45_r3,TrkA_PI3Kinh_0_r1,...,TrkB_DMSO_45_r6,TrkB_MEKinh_45_r4,TrkB_MEKinh_45_r5,TrkB_MEKinh_45_r6,TrkB_AKTinh_45_r4,TrkB_AKTinh_45_r5,TrkB_AKTinh_45_r6,TrkB_JNK8inh_45_r4,TrkB_JNK8inh_45_r5,TrkB_JNK8inh_45_r6
Antibodies,,,,,,,,,,,,,,,,,,,,,
Bad P Ser136,1.0,1.0,1.0,1.195214,1.024246,1.260908,1.195922,0.865755,1.214001,0.495048,...,0.877275,1.057239,0.353452,0.748506,0.509588,0.375864,0.477642,0.908119,0.461701,0.861967
Bad P Ser112,1.0,1.0,1.0,1.501239,0.879089,1.146981,1.559765,1.306904,1.135425,0.482351,...,1.812681,1.277119,1.190415,1.504822,1.198173,1.283662,1.483497,1.309783,1.251183,1.834155
Bak,1.0,1.0,1.0,1.291721,1.079495,1.282790,1.357286,1.165838,1.193006,1.082495,...,0.816592,0.951201,1.111161,0.886745,1.169466,1.242868,0.615858,0.739281,1.019806,0.925118
CrkL P Tyr207,1.0,1.0,1.0,1.338888,0.977276,1.270155,1.327561,1.082845,1.360543,1.141267,...,2.869546,1.574401,1.791657,2.302706,1.545489,1.999032,2.138966,1.568556,1.802026,2.460437
HSP27 (HSPB1) P Ser78,1.0,1.0,1.0,1.079592,1.207202,1.415305,1.163609,1.182375,1.090034,1.979697,...,3.228579,3.544088,3.135005,2.597842,4.490556,4.201606,2.841237,4.812367,3.319634,3.931694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HSP27 (HSPB1),1.0,1.0,1.0,1.052418,0.965690,1.142180,1.051292,1.042704,1.087223,2.039877,...,1.258354,0.905907,0.895537,1.066353,0.619436,0.823818,0.729085,0.923475,0.932069,1.303631
Ras,1.0,1.0,1.0,1.163388,0.939451,1.139635,1.165525,1.033746,1.061109,1.184621,...,0.809489,0.952662,0.900351,0.958077,0.969699,0.929930,0.807424,1.111880,0.854320,0.834037
Stat1,1.0,1.0,1.0,1.134809,0.915523,1.184589,1.166527,0.987451,0.974659,0.822829,...,0.748369,0.985693,0.838139,0.785082,0.920244,0.997224,0.602551,1.037386,1.070708,0.591864


In [6]:
# filling NaNs
TPs = [0,10,45]
for cell in cells:
    for timep in TPs:
        for replicate in range(replicates_DMSO):
            str_n=cell + '_DMSO_' + str(timep) + '_r' + str(replicate+1)
            if np.isnan(RPPA_total_norm[str_n].loc['pTrk']):
                str_reg = cell + '_DMSO_' + str(timep) + '_r.*'
                RPPA_total_norm[str_n].loc['pTrk'] = RPPA_total_norm.loc['pTrk'].filter(regex=str_reg).mean(axis=0,skipna=True)
                #print(RPPA_total_norm.loc['pTrk'].filter(regex=str_reg).mean(axis=0,skipna=True))

controls = RPPA_total_norm.filter(regex='.*DMSO.*')
controls

,TrkA_DMSO_0_r1,TrkA_DMSO_0_r2,TrkA_DMSO_0_r3,TrkA_DMSO_10_r1,TrkA_DMSO_10_r2,TrkA_DMSO_10_r3,TrkA_DMSO_45_r1,TrkA_DMSO_45_r2,TrkA_DMSO_45_r3,TrkB_DMSO_0_r1,...,TrkA_DMSO_45_r6,TrkB_DMSO_0_r4,TrkB_DMSO_0_r5,TrkB_DMSO_0_r6,TrkB_DMSO_10_r4,TrkB_DMSO_10_r5,TrkB_DMSO_10_r6,TrkB_DMSO_45_r4,TrkB_DMSO_45_r5,TrkB_DMSO_45_r6
Antibodies,,,,,,,,,,,,,,,,,,,,,
Bad P Ser136,1.0,1.0,1.0,1.195214,1.024246,1.260908,1.195922,0.865755,1.214001,1.0,...,1.153764,1.0,1.0,1.0,0.911601,1.092188,0.857805,0.504786,0.456516,0.877275
Bad P Ser112,1.0,1.0,1.0,1.501239,0.879089,1.146981,1.559765,1.306904,1.135425,1.0,...,1.168979,1.0,1.0,1.0,1.266108,0.958668,1.230827,1.093156,1.230254,1.812681
Bak,1.0,1.0,1.0,1.291721,1.079495,1.282790,1.357286,1.165838,1.193006,1.0,...,0.945322,1.0,1.0,1.0,1.480998,1.403273,0.983929,0.901873,0.753822,0.816592
CrkL P Tyr207,1.0,1.0,1.0,1.338888,0.977276,1.270155,1.327561,1.082845,1.360543,1.0,...,1.071355,1.0,1.0,1.0,1.496703,2.282126,1.965369,1.308922,1.878024,2.869546
HSP27 (HSPB1) P Ser78,1.0,1.0,1.0,1.079592,1.207202,1.415305,1.163609,1.182375,1.090034,1.0,...,0.803627,1.0,1.0,1.0,6.373076,4.754273,3.391033,3.135229,3.251377,3.228579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HSP27 (HSPB1),1.0,1.0,1.0,1.052418,0.965690,1.142180,1.051292,1.042704,1.087223,1.0,...,1.037670,1.0,1.0,1.0,0.980254,0.976628,1.051677,0.766519,0.927305,1.258354
Ras,1.0,1.0,1.0,1.163388,0.939451,1.139635,1.165525,1.033746,1.061109,1.0,...,0.856522,1.0,1.0,1.0,1.135871,0.906469,0.817583,0.823354,0.863183,0.809489
Stat1,1.0,1.0,1.0,1.134809,0.915523,1.184589,1.166527,0.987451,0.974659,1.0,...,0.968772,1.0,1.0,1.0,1.435843,0.963965,0.937806,0.968209,1.074635,0.748369


In [7]:
#building STV

controls = RPPA_total_norm.filter(regex='.*DMSO_[^0].*')
cls = [0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1]

svc = SVC(kernel='linear')
svc.fit(controls.transpose(),cls)

# here is our STV
TrkA_center = RPPA_total_norm.filter(regex='TrkA.*DMSO_[^0].*').sum(axis=1) / len(RPPA_total_norm.filter(regex='TrkA.*DMSO_[^0].*').columns)
TrkB_center = RPPA_total_norm.filter(regex='TrkB.*DMSO_[^0].*').sum(axis=1) / len(RPPA_total_norm.filter(regex='TrkB.*DMSO_[^0].*').columns)
n_vec = svc.coef_[0]/linalg.norm(svc.coef_[0])
norm_vec = (TrkB_center - TrkA_center)/linalg.norm(TrkB_center - TrkA_center)
RPPA_total_norm.insert(0,"norm_vec",norm_vec)
# plus sign means propensity towards TrkA state
# minus sign means propensity towards TrkB state (e.g. ppERK or pAKT)

# sort antibodies by contribution to STV
RPPA_total_norm.sort_values(by='norm_vec')
#RPPA_total_norm.to_csv("STV ranking.csv")

,norm_vec,TrkA_DMSO_0_r1,TrkA_DMSO_0_r2,TrkA_DMSO_0_r3,TrkA_DMSO_10_r1,TrkA_DMSO_10_r2,TrkA_DMSO_10_r3,TrkA_DMSO_45_r1,TrkA_DMSO_45_r2,TrkA_DMSO_45_r3,...,TrkB_DMSO_45_r6,TrkB_MEKinh_45_r4,TrkB_MEKinh_45_r5,TrkB_MEKinh_45_r6,TrkB_AKTinh_45_r4,TrkB_AKTinh_45_r5,TrkB_AKTinh_45_r6,TrkB_JNK8inh_45_r4,TrkB_JNK8inh_45_r5,TrkB_JNK8inh_45_r6
Antibodies,,,,,,,,,,,,,,,,,,,,,
Caspase 3 cleaved,-0.063582,1.0,1.0,1.0,1.993175,1.173172,1.566501,2.137101,1.118907,1.120055,...,1.065722,1.274657,1.509392,0.741371,1.321198,1.771062,0.936652,0.525893,1.391759,0.944195
Src,-0.026383,1.0,1.0,1.0,1.141311,0.930196,1.242927,1.036205,0.952202,1.039633,...,0.592791,0.742338,0.589300,0.602478,0.721397,0.646385,0.583933,0.850282,0.713769,0.687086
Bad P Ser136,-0.024256,1.0,1.0,1.0,1.195214,1.024246,1.260908,1.195922,0.865755,1.214001,...,0.877275,1.057239,0.353452,0.748506,0.509588,0.375864,0.477642,0.908119,0.461701,0.861967
PKC-zeta,-0.021307,1.0,1.0,1.0,1.207725,1.026568,1.465118,1.164160,1.155453,1.237001,...,1.694520,1.357598,0.399973,0.855702,0.948719,0.654343,1.094418,0.653891,0.606135,1.193897
Raf P Ser259,-0.020970,1.0,1.0,1.0,1.135425,0.929860,1.153372,1.302875,1.091787,1.205705,...,1.168623,0.808721,0.718761,0.980193,0.845761,0.803409,1.104386,0.868741,0.786874,1.269732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"p44/42 MAPK (ERK1/2) P Thr202/Thr185,Tyr204/Tyr187",0.286083,1.0,1.0,1.0,4.848076,3.810726,4.913487,4.099184,3.507983,3.842838,...,6.593423,4.250916,4.700073,4.572757,4.888758,5.905713,5.294073,4.999564,5.676591,6.174600
Akt P Ser473,0.331994,1.0,1.0,1.0,1.412401,1.196654,1.833086,1.753096,1.751853,2.159915,...,6.087380,5.667397,4.951005,5.633737,1.570201,1.651556,1.625553,5.725874,5.166922,5.786203
Akt P Thr308,0.340340,1.0,1.0,1.0,1.946301,1.645936,2.211320,2.210482,1.997042,2.449424,...,7.345179,5.169720,4.662878,4.565637,1.442918,1.819946,1.960745,5.191764,4.775045,6.211890


In [8]:
angle = (np.arccos(np.dot(norm_vec,n_vec))/np.pi)*180
angle

21.58040158664776

In [9]:
# delete pTrk row
#RPPA_total_norm = RPPA_total_norm.drop(index="pTrk")

In [10]:
# now calculating dot products between perturbation vectors and STV without taking into account data on RTK and perturbed modules

# assigining outputs to different modules
RTK_module_names = ['ErbB-2/Her2/EGFR P Tyr1248/Tyr1173','ErbB-3/Her3/EGFR P Tyr1289','EGFR P Tyr1173','Met P Tyr1234','Met P Tyr1349','IGF-1R beta P Tyr1162,Tyr1163','EGFR P Tyr1173','EGFR P Tyr1086','PDGFR P Tyr1021','PDGFR  P Tyr751','ErbB-1/EGFR','ErbB-3/Her3/EGFR','Met','IGF-1R beta']
#RTK_module_output = ['ErbB-2/Her2/EGFR P Tyr1248/Tyr1173','ErbB-3/Her3/EGFR P Tyr1289','EGFR P Tyr1173','Met P Tyr1234','Met P Tyr1349','IGF-1R beta P Tyr1162,Tyr1163','EGFR P Tyr1173','EGFR P Tyr1086','PDGFR P Tyr1021','PDGFR  P Tyr751']
#RTK_module_output_diag = ['ErbB-2/Her2/EGFR P Tyr1248/Tyr1173','ErbB-3/Her3/EGFR P Tyr1289','EGFR P Tyr1173','Met P Tyr1234','Met P Tyr1349','IGF-1R beta P Tyr1162,Tyr1163','EGFR P Tyr1173','EGFR P Tyr1086','PDGFR P Tyr1021','PDGFR  P Tyr751']
RTK_module_output = ['ErbB-2/Her2/EGFR P Tyr1248/Tyr1173']
RTK_module_output_diag = ['ErbB-2/Her2/EGFR P Tyr1248/Tyr1173']

ERK_module_names = ['MEK1/2 P Ser217/221','p44/42 MAPK (ERK1/2) P Thr202/Thr185,Tyr204/Tyr187','Raf P Ser259','Raf P Ser338','p44/42 MAPK (ERK1/2)','Raf P Ser338','B-Raf [EP152Y]','Ras']
ERK_module_output = ['p44/42 MAPK (ERK1/2) P Thr202/Thr185,Tyr204/Tyr187']
ERK_module_output_diag = ['p44/42 MAPK (ERK1/2) P Thr202/Thr185,Tyr204/Tyr187']

AKT_module_names = ['PI3 Kinase p110-alpha','IRS-1 P S636/639','Akt P Thr308','Akt P Ser473','Akt','PTEN','PTEN P Ser380,Thr382,Thr383']
#AKT_module_output = ['Akt P Thr308','Akt P Ser473']
AKT_module_output = ['Akt P Ser473']
AKT_module_output_diag = ['Akt P Ser473']

JNK_module_names = ['SAPK/JNK P Thr183,Tyr185','SAPK/JNK (JNK2)']
JNK_module_output = ['SAPK/JNK P Thr183,Tyr185']
JNK_module_output_diag = ['c-Jun P Ser73']

#S6K_module_names = ['p70 S6 Kinase P Thr389','p70 S6 Kinase P Thr421,Ser424','p70 S6 Kinase']
S6K_module_names = ['p70 S6 Kinase P Thr389','p70 S6 Kinase P Thr421,Ser424','p70 S6 Kinase','mTOR P Ser2448','mTOR','Tsc-2 (Tuberin) P Thr1462','Tsc-2 (Tuberin)','S6 Ribosomal Protein']
#S6K_module_output = ['p70 S6 Kinase P Thr389','p70 S6 Kinase P Thr421,Ser424']
S6K_module_output = ['p70 S6 Kinase P Thr389']
S6K_module_output_diag = ['S6 Ribosomal protein P Ser235,Ser236']

RSK_module_names = ['p90 S6 kinase (Rsk1-3) P Thr359,Ser363','Rsk2 Pser 227']
#RSK_module_output = ['p90 S6 kinase (Rsk1-3) P Thr359,Ser363','Rsk2 Pser 227']
RSK_module_output = ['Rsk2 Pser 227']
RSK_module_output_diag = ['Rsk2 Pser 227','Bad P Ser112']

#p38_module_names = ['p38 MAPK PThr180,Tyr182','p38 MAPK']
p38_module_names = ['p38 MAPK PThr180,Tyr182','p38 MAPK','HSP27 (HSPB1) P Ser78','MAPKAPK-2 P Thr334']
#total_proteins = ['Stat1','HSP27 (HSPB1)','CrkL','Mouse_1','Mouse_2','Mouse_3','PKC-alpha','RhoA (67BC)','YAP1 [EP1674Y]','Rock1 (C8F7)','Stat3','CDK2 ','PLC-gamma1','GSK-3-beta','Caspase 3','Src ','Stat5','JAK1','c-Myc','PLC-gamma1 P Tyr783','Rb','PDK-1','beta-Catenin','p53','Stat1 P Tyr701']
#total_proteins = ['IRS-1 P S636/639','Mouse_1','Mouse_2','Mouse_3','Stat1 P Tyr701','PLC-gamma1 P Tyr783']
total_proteins = ['pTrk','IRS-1 P S636/639','Mouse_1','Mouse_2','Mouse_3','PLC-gamma1 P Tyr783']

treatments_output = ["JNKinh","MEKinh","AKTinh","JNK8inh","S6Kinh","RSKinh"]
treatments_output_Trk = ["JNKinh","MEKinh","AKTinh"]
modules_output = [ERK_module_output,AKT_module_output,JNK_module_output,S6K_module_output,RSK_module_output,RTK_module_output]
modules_output_diag = [ERK_module_output_diag,AKT_module_output_diag,JNK_module_output_diag,S6K_module_output_diag,RSK_module_output_diag,RTK_module_output_diag]

total_names = RTK_module_names + ERK_module_names + AKT_module_names + S6K_module_names + RSK_module_names + JNK_module_names + p38_module_names + total_proteins
print(total_names)

['ErbB-2/Her2/EGFR P Tyr1248/Tyr1173', 'ErbB-3/Her3/EGFR P Tyr1289', 'EGFR P Tyr1173', 'Met P Tyr1234', 'Met P Tyr1349', 'IGF-1R beta P Tyr1162,Tyr1163', 'EGFR P Tyr1173', 'EGFR P Tyr1086', 'PDGFR P Tyr1021', 'PDGFR  P Tyr751', 'ErbB-1/EGFR', 'ErbB-3/Her3/EGFR', 'Met', 'IGF-1R beta', 'MEK1/2 P Ser217/221', 'p44/42 MAPK (ERK1/2) P Thr202/Thr185,Tyr204/Tyr187', 'Raf P Ser259', 'Raf P Ser338', 'p44/42 MAPK (ERK1/2)', 'Raf P Ser338', 'B-Raf [EP152Y]', 'Ras', 'PI3 Kinase p110-alpha', 'IRS-1 P S636/639', 'Akt P Thr308', 'Akt P Ser473', 'Akt', 'PTEN', 'PTEN P Ser380,Thr382,Thr383', 'p70 S6 Kinase P Thr389', 'p70 S6 Kinase P Thr421,Ser424', 'p70 S6 Kinase', 'mTOR P Ser2448', 'mTOR', 'Tsc-2 (Tuberin) P Thr1462', 'Tsc-2 (Tuberin)', 'S6 Ribosomal Protein', 'p90 S6 kinase (Rsk1-3) P Thr359,Ser363', 'Rsk2 Pser 227', 'SAPK/JNK P Thr183,Tyr185', 'SAPK/JNK (JNK2)', 'p38 MAPK PThr180,Tyr182', 'p38 MAPK', 'HSP27 (HSPB1) P Ser78', 'MAPKAPK-2 P Thr334', 'pTrk', 'IRS-1 P S636/639', 'Mouse_1', 'Mouse_2', 'M

In [11]:
RPPA_total_norm_red = RPPA_total_norm.drop(index=total_names)
controls_red = RPPA_total_norm_red.filter(regex='.*DMSO_45.*')
#controls_red.columns

In [12]:
#building new STV

cls_red = [0,0,0,1,1,1,0,0,0,1,1,1]

svc_red = SVC(kernel='linear')
svc_red.fit(controls_red.transpose(),cls_red)
b_red = svc_red.intercept_[0]
n_vec_red = svc_red.coef_[0]/linalg.norm(svc_red.coef_[0])

# here is our STV
#norm_vec_red = - svc_red.coef_[0]/linalg.norm(svc_red.coef_[0])
TrkA_center = RPPA_total_norm_red.filter(regex='TrkA.*DMSO_45.*').sum(axis=1) / len(RPPA_total_norm_red.filter(regex='TrkA.*DMSO_45.*').columns)
TrkB_center = RPPA_total_norm_red.filter(regex='TrkB.*DMSO_45.*').sum(axis=1) / len(RPPA_total_norm_red.filter(regex='TrkB.*DMSO_45.*').columns)
norm_vec_red = (TrkB_center - TrkA_center)/linalg.norm(TrkB_center - TrkA_center)
del RPPA_total_norm_red["norm_vec"]
RPPA_total_norm_red.insert(0,"norm_vec",norm_vec_red)
# plus sign means propensity towards TrkA state
# minus sign means propensity towards TrkB state (e.g. ppERK or pAKT)

# sort antibodies by contribution to STV
RPPA_total_norm_red.sort_values(by='norm_vec')

,norm_vec,TrkA_DMSO_0_r1,TrkA_DMSO_0_r2,TrkA_DMSO_0_r3,TrkA_DMSO_10_r1,TrkA_DMSO_10_r2,TrkA_DMSO_10_r3,TrkA_DMSO_45_r1,TrkA_DMSO_45_r2,TrkA_DMSO_45_r3,...,TrkB_DMSO_45_r6,TrkB_MEKinh_45_r4,TrkB_MEKinh_45_r5,TrkB_MEKinh_45_r6,TrkB_AKTinh_45_r4,TrkB_AKTinh_45_r5,TrkB_AKTinh_45_r6,TrkB_JNK8inh_45_r4,TrkB_JNK8inh_45_r5,TrkB_JNK8inh_45_r6
Antibodies,,,,,,,,,,,,,,,,,,,,,
Caspase 3 cleaved,-0.127765,1.0,1.0,1.0,1.993175,1.173172,1.566501,2.137101,1.118907,1.120055,...,1.065722,1.274657,1.509392,0.741371,1.321198,1.771062,0.936652,0.525893,1.391759,0.944195
Src,-0.049237,1.0,1.0,1.0,1.141311,0.930196,1.242927,1.036205,0.952202,1.039633,...,0.592791,0.742338,0.589300,0.602478,0.721397,0.646385,0.583933,0.850282,0.713769,0.687086
PKC-zeta,-0.038672,1.0,1.0,1.0,1.207725,1.026568,1.465118,1.164160,1.155453,1.237001,...,1.694520,1.357598,0.399973,0.855702,0.948719,0.654343,1.094418,0.653891,0.606135,1.193897
Rock1 (C8F7),-0.036274,1.0,1.0,1.0,1.107459,0.985061,1.092401,1.204258,1.065882,1.258269,...,0.672961,0.903343,1.285445,0.505167,0.943344,1.546696,0.554718,1.116135,1.346212,0.554742
PDK-1,-0.036025,1.0,1.0,1.0,1.193055,0.917105,1.212314,1.268578,0.983393,1.122278,...,0.868919,0.882720,0.741165,0.821387,0.729251,0.828327,0.690100,0.919081,0.687826,0.848902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CrkL P Tyr207,0.115442,1.0,1.0,1.0,1.338888,0.977276,1.270155,1.327561,1.082845,1.360543,...,2.869546,1.574401,1.791657,2.302706,1.545489,1.999032,2.138966,1.568556,1.802026,2.460437
PKC (pan) P Ser660 (beta-2),0.125741,1.0,1.0,1.0,1.421199,1.381788,1.591265,1.578156,1.680934,1.636342,...,2.887600,2.500598,2.089772,2.186185,2.564976,2.680059,2.207372,2.666394,2.278826,2.845704
cdc25A,0.160390,1.0,1.0,1.0,1.923830,1.235878,2.044709,1.632886,1.468950,1.562057,...,2.381021,2.004661,4.557915,1.653233,1.945179,4.549870,1.776065,2.059524,4.265267,2.048395


In [13]:
angle_red = (np.arccos(np.dot(norm_vec_red,n_vec_red))/np.pi)*180
angle_red

41.43179044955859

In [14]:
# summary

# full dataset:
# the dataset itself (samples are in columns and analytes are in rows): controls
# labeling of the samples: cls
# SVM model: svc
# SVM model was created by the following command: svc.fit(controls.transpose(),cls)
# normal to the separation hyperplane (length = 1): n_vec
# vector conneting centroids (length = 1): norm_vec
# angle between these vectors: angle

# reduced dataset (all analytes minus core network related analytes):
# the dataset itself (samples in columns and analytes in rows): controls_red
# labeling of the samples: cls_red
# SVM model: svc_red
# SVM model was created by the following command: svc_red.fit(controls_red.transpose(),cls_red)
# normal to the separation hyperplane (length = 1): n_vec_red
# vector conneting centroids (length = 1): norm_vec_red
# angle between these vectors: angle_red


In [15]:
controls.to_csv('rppa_controls.csv')
controls

,TrkA_DMSO_10_r1,TrkA_DMSO_10_r2,TrkA_DMSO_10_r3,TrkA_DMSO_45_r1,TrkA_DMSO_45_r2,TrkA_DMSO_45_r3,TrkB_DMSO_10_r1,TrkB_DMSO_10_r2,TrkB_DMSO_10_r3,TrkB_DMSO_45_r1,...,TrkA_DMSO_10_r6,TrkA_DMSO_45_r4,TrkA_DMSO_45_r5,TrkA_DMSO_45_r6,TrkB_DMSO_10_r4,TrkB_DMSO_10_r5,TrkB_DMSO_10_r6,TrkB_DMSO_45_r4,TrkB_DMSO_45_r5,TrkB_DMSO_45_r6
Antibodies,,,,,,,,,,,,,,,,,,,,,
Bad P Ser136,1.195214,1.024246,1.260908,1.195922,0.865755,1.214001,1.144136,0.706490,0.867280,1.041495,...,0.993708,0.662678,0.676663,1.153764,0.911601,1.092188,0.857805,0.504786,0.456516,0.877275
Bad P Ser112,1.501239,0.879089,1.146981,1.559765,1.306904,1.135425,1.660879,1.267973,1.151540,2.201293,...,1.067798,1.014344,0.868698,1.168979,1.266108,0.958668,1.230827,1.093156,1.230254,1.812681
Bak,1.291721,1.079495,1.282790,1.357286,1.165838,1.193006,1.277383,1.062855,0.904617,1.062596,...,1.095152,0.731157,0.695485,0.945322,1.480998,1.403273,0.983929,0.901873,0.753822,0.816592
CrkL P Tyr207,1.338888,0.977276,1.270155,1.327561,1.082845,1.360543,3.005067,2.288406,2.117516,2.838728,...,0.934763,1.491168,0.829043,1.071355,1.496703,2.282126,1.965369,1.308922,1.878024,2.869546
HSP27 (HSPB1) P Ser78,1.079592,1.207202,1.415305,1.163609,1.182375,1.090034,0.947987,0.947915,0.794166,1.436634,...,0.777025,1.011703,1.255780,0.803627,6.373076,4.754273,3.391033,3.135229,3.251377,3.228579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HSP27 (HSPB1),1.052418,0.965690,1.142180,1.051292,1.042704,1.087223,0.981662,0.775537,0.800970,0.993965,...,0.783901,0.789410,0.983441,1.037670,0.980254,0.976628,1.051677,0.766519,0.927305,1.258354
Ras,1.163388,0.939451,1.139635,1.165525,1.033746,1.061109,0.986838,0.870579,1.075022,0.984890,...,0.926491,0.734695,1.267359,0.856522,1.135871,0.906469,0.817583,0.823354,0.863183,0.809489
Stat1,1.134809,0.915523,1.184589,1.166527,0.987451,0.974659,1.046355,0.925302,1.063131,0.963030,...,0.977786,0.849016,1.080012,0.968772,1.435843,0.963965,0.937806,0.968209,1.074635,0.748369


In [16]:
controls_red.to_csv('rppa_controls_red.csv')
controls_red

,TrkA_DMSO_45_r1,TrkA_DMSO_45_r2,TrkA_DMSO_45_r3,TrkB_DMSO_45_r1,TrkB_DMSO_45_r2,TrkB_DMSO_45_r3,TrkA_DMSO_45_r4,TrkA_DMSO_45_r5,TrkA_DMSO_45_r6,TrkB_DMSO_45_r4,TrkB_DMSO_45_r5,TrkB_DMSO_45_r6
Antibodies,,,,,,,,,,,,
Bad P Ser136,1.195922,0.865755,1.214001,1.041495,0.695469,0.667934,0.662678,0.676663,1.153764,0.504786,0.456516,0.877275
Bad P Ser112,1.559765,1.306904,1.135425,2.201293,1.420995,1.379187,1.014344,0.868698,1.168979,1.093156,1.230254,1.812681
Bak,1.357286,1.165838,1.193006,1.062596,0.935455,0.877001,0.731157,0.695485,0.945322,0.901873,0.753822,0.816592
CrkL P Tyr207,1.327561,1.082845,1.360543,2.838728,1.926664,1.906306,1.491168,0.829043,1.071355,1.308922,1.878024,2.869546
JAK1,1.297287,0.846140,1.387782,1.047925,0.987803,1.081421,0.897964,1.359162,0.406112,1.196035,1.089896,0.838465
...,...,...,...,...,...,...,...,...,...,...,...,...
p21 CIP/WAF1,1.106405,1.082039,0.964859,0.856260,0.794351,0.840668,0.907715,1.166843,1.144468,0.869556,1.074695,1.187784
Stat3 P Tyr705,1.244418,1.050401,1.152010,0.925106,0.800083,0.898269,0.854592,0.880204,0.894033,0.795313,0.910637,0.983837
CrkL,1.045429,0.845945,0.981257,0.935369,0.782709,0.834232,0.836015,1.051350,0.992255,0.701424,0.846179,1.156897
